In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tempfile import TemporaryDirectory as TD
import subprocess as sb
import os.path as osp

In [3]:
EVAL_PTH = '../src/'
import sys
if EVAL_PTH not in sys.path:
    sys.path.append(EVAL_PTH)
import eval as ev

In [5]:
data = dict(sp = dict( path = '/gstore/data/resbioai/spacerangers/cci-explore/pipeline/example_data/qx_visified.h5ad'),
            sc = dict(path = '/gstore/data/resbioai/spacerangers/cci-explore/pipeline/example_data/qx_mappable.h5ad'),
            )

In [6]:
input_data = ev.ut.read_data(data)

In [7]:
input_data

{'X_to': AnnData object with n_obs × n_vars = 4125 × 313
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'visify_cell_map_mp', 'visify_cell_map_og', 'visify_params'
     obsm: 'spatial'
     layers: 'downsampled_0.85_1.00', 'mul_noise_0.50',
 'X_from': AnnData object with n_obs × n_vars = 39945 × 313
     obs: 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'visify_cell_map'
     obsm: 'spatial'}

In [8]:
tg_out = ev.map_methods.TangramV1Map.run(input_data,num_epochs = 2, return_hard = False)


INFO:root:313 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:313 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 313 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.473, KL reg: 0.338


INFO:root:Saving results..


In [9]:
with TD() as tmpdir:
    ev.map_methods.TangramV1Map.save(tg_out,tmpdir, save_keys = 'S_from')
    sb.run(['ls',tmpdir])
    sb.run(['head', osp.join(tmpdir,'S_from.csv')])

S_from.csv
,x,y
12,3651.494047865344,2678.211987061607
13,3649.687063449129,2668.187179181118
17,3681.1239979106845,2753.3752782266347
21,3666.3642388721973,2721.9014602085363
28,3634.871158238422,2763.521015301578
31,3771.0768539008996,2764.984513107409
34,3697.92510579573,2706.925388308326
35,3733.303687830542,2695.8860686300363
39,3611.817661254977,2732.60885256376


In [10]:
input_data.update(tg_out)

In [11]:
pr_out = ev.pred_methods.TangramV1Pred.run(input_data)

In [ ]:
with TD() as tmpdir:
    ev.pred_methods.TangramV1Pred.save(pr_out,tmpdir)
    sb.run(['head', osp.join(tmpdir,'X_to_pred.csv'),])

In [13]:
input_data.update(pr_out)

25,5.3126845,1.4504654,0.9961734,0.035132162,7.6921816,9.849103,4.6933494,6.3285604,4.0711293,11.56148,6.5688953


In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
input_data['X_from'].obs['label'] = np.random.choice(['A','B','C'],
                                                     replace = True, 
                                                     size = input_data['X_from'].shape[0])

In [138]:
grp_out = ev.grp.ThresholdGroup.run(input_data,
                                    feature_name='abcc11',
                                    thres_x = 9,
                                    thres_t = 0.5,
                                    add_covariates = {'from':'label'})

In [139]:
grp_out['D_from']

,low_abcc11,high_abcc11,A,B,C
12,0,1,0,1,0
13,0,1,0,1,0
17,0,1,0,1,0
21,0,1,0,1,0
28,0,1,0,0,1
...,...,...,...,...,...
167758,0,1,0,1,0
167763,0,1,0,1,0
167765,0,1,0,1,0
167769,0,1,0,1,0


A bit more customization:

In [18]:
grp_out = ev.grp.ThresholdGroup.run(input_data,
                                    feature_name='abcc11',
                                    thres_x = 9,
                                    covariates = {'from':'label'},
                                    subset = 'A',
                                    merge = True,
                                   )

In [19]:
grp_out['D_from']

,low_abcc11_A,high_abcc11_A
12,0,0
13,0,0
17,0,0
21,0,0
28,0,0
...,...,...
167758,0,0
167763,0,0
167765,0,0
167769,0,0
